In [1]:
API = "Your API Key"
api_url = "https://api.corona-zahlen.org/germany"

In [8]:
import logging
import requests
from telegram import Update, ForceReply
from telegram.ext import Updater, CommandHandler, MessageHandler, Filters, CallbackContext
import datetime
import pytz

#code based on examples on https://github.com/python-telegram-bot/python-telegram-bot/blob/master/examples/README.md

In [10]:
logging.basicConfig(
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s', level=logging.INFO
)

logger = logging.getLogger(__name__)


# Define a few command handlers. These usually take the two arguments update and
# context.
def start(update: Update, _: CallbackContext) -> None:
    """Send a message when the command /start is issued."""
    user = update.effective_user
    update.message.reply_markdown_v2(
        fr'Hi {user.mention_markdown_v2()}\!',
        reply_markup=ForceReply(selective=True),
    )
    update.message.reply_text("/set - to set notification regarding emergency brake.\n/unset - stop notofications")

# polling incidence data
def incidence(update: Update, _: CallbackContext) -> None:
    incidence_n = requests.get(url=api_url).json()["weekIncidence"]
    update.message.reply_text(f"7-days-incidence-number: {incidence_n}")
    

# notification part
def alarm(context: CallbackContext) -> None:
    """Send the alarm message."""
    
    past_7 = requests.get(url=api_url+"/history/incidence/7").json()


    def check_business_day(date):
        d = datetime.datetime.strptime(date,"%Y-%m-%dT%H:%M:%S.%fZ")
        judgement = False
        if d.isoweekday() in range(1, 6):
            judgement = True

        return judgement

    p_b = [i["weekIncidence"] for i in past_7["data"] if check_business_day(i["date"]) == True]
    p_l = [i for i in p_b if i <= 100]

    if len(p_l) == 5:
        test = "Emergency brake lockdown is gonna end, yay!"
        job = context.job
        context.bot.send_message(job.context, text=test)



def remove_job_if_exists(name: str, context: CallbackContext) -> bool:
    """Remove job with given name. Returns whether job was removed."""
    current_jobs = context.job_queue.get_jobs_by_name(name)
    if not current_jobs:
        return False
    for job in current_jobs:
        job.schedule_removal()
    return True

def set_timer(update: Update, context: CallbackContext) -> None:
    """Add a job to the queue."""
    chat_id = update.message.chat_id
    try:
        due = datetime.time(hour=17, minute=0, tzinfo=pytz.timezone('Asia/Kolkata'))
        job_removed = remove_job_if_exists(str(chat_id), context)
        context.job_queue.run_daily(alarm, due, context=chat_id, name=str(chat_id))

        text = 'Notification successfully set!'
        if job_removed:
            text += ' Old one was removed.'
        update.message.reply_text(text)

    except (IndexError, ValueError):
        update.message.reply_text('Usage: /set <seconds>')


def unset(update: Update, context: CallbackContext) -> None:
    """Remove the job if the user changed their mind."""
    chat_id = update.message.chat_id
    job_removed = remove_job_if_exists(str(chat_id), context)
    text = 'Notification successfully cancelled!' if job_removed else 'You have no active notification.'
    update.message.reply_text(text)


def main() -> None:
    """Start the bot."""
    # Create the Updater and pass it your bot's token.
    updater = Updater(API)

    # Get the dispatcher to register handlers
    dispatcher = updater.dispatcher

    # on different commands - answer in Telegram
    dispatcher.add_handler(CommandHandler("start", start))
    dispatcher.add_handler(CommandHandler("incidence", incidence))
    dispatcher.add_handler(CommandHandler("set", set_timer))
    dispatcher.add_handler(CommandHandler("unset", unset))

    # Start the Bot
    updater.start_polling()

    # Run the bot until you press Ctrl-C or the process receives SIGINT,
    # SIGTERM or SIGABRT. This should be used most of the time, since
    # start_polling() is non-blocking and will stop the bot gracefully.
    updater.idle()


if __name__ == '__main__':
    main()

2021-05-20 13:33:09,879 - apscheduler.scheduler - INFO - Scheduler started
2021-05-20 13:33:10,179 - apscheduler.scheduler - INFO - Added job "134635594" to job store "default"
2021-05-20 13:33:23,137 - telegram.ext.updater - INFO - Received signal 2 (SIGINT), stopping...
2021-05-20 13:33:23,138 - apscheduler.scheduler - INFO - Scheduler has been shut down
